In [2]:
from pathlib import Path 
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd
from collections import OrderedDict
import sys
import os
import seaborn as sns
import researchpy as rp
import statsmodels.formula.api as smf
import scipy.stats as stats
import ipympl

#sys.path.append('/Users/alina/Desktop/MIT/code/ADHD/MTA/helper')
from helper import rr, prep, var_dict

%load_ext autoreload
%autoreload 2

In [3]:
data_root = '/Volumes/Samsung_T5/MIT/mta'
data_derived = '/Volumes/Samsung_T5/MIT/mta/output/derived_data'
os.listdir(data_derived)

['snap_baseline_p_t.csv',
 'snap_14_p_t.csv',
 'snap_24_p_t.csv',
 'snap_36_p_t.csv',
 'ssrs_baseline_p_t.csv',
 'ssrs_14_p_t.csv',
 'ssrs_24_p_t.csv',
 'ssrs_36_p_t.csv',
 'masc_c_treat.csv',
 'pc_p_treat.csv',
 'wechsler_c_treat.csv',
 'snap_p_t.csv',
 'ssrs_p_t.csv',
 'masc_baseline.csv',
 'masc_14.csv',
 'masc_24.csv',
 'masc_36.csv',
 'pc_p_baseline.csv',
 'pc_p_14.csv',
 'pc_p_24.csv',
 'pc_p_36.csv',
 'wechsler_baseline.csv',
 'wechsler_14.csv',
 'wechsler_24.csv',
 'wechsler_36.csv',
 'masc_c.csv',
 'pc_p.csv',
 'wechsler_c.csv',
 'treatment_groups.csv',
 'comorb.csv']

### Mediators 
- prior treatment with psychoactive medication, 
- ODD/CD comorbidity  (parent, DIS version 3.0), 
- comorbid anx (parent, DIS version 2.3), 
- familial reciept of public assistance
### Moderators 
- as intended (M) : 
  - acceptance of treatment 
  - family attendance of 80% monthly medication visit 
  - prescription written and deluvered to family at visit 
- below intended 
  - refusal, stop treatment, moving, droping out prior to 80% of visits, or attendance below cutoff 
- as intended (B) : 
  - acceptance of treatment 
  - attendance 75% group parent training sessions 
  - child attendance 74% summer treatment program 
  - child + paraproffessional present 75% in classroom at intended days 
- below intended :
  - refusal of any treatment component, 
  - low attendance, 
  - dicontinuation before cutoffs 
- as intended (C): 
  - all criteria for M and B 

In [93]:
trt_name = pd.read_csv(Path(data_derived, 'treatment_groups.csv'))
services_file = 'scapijus01.txt'
meds_file = 'medse01.txt'
expectations_file = 'expect01.txt'
initial_percieved_satisfaction_file = 'debrief01.txt'
int_dsm4_file = 'scid01.txt'
diags1_file  = 'diagpsx01.txt'


services = pd.read_csv(Path(data_root, services_file), delimiter='\t')
meds = pd.read_csv(Path(data_root, meds_file), delimiter='\t')

dsm4 = pd.read_csv(Path(data_root, int_dsm4_file), delimiter='\t')
diags1 = pd.read_csv(Path(data_root, diags1_file), delimiter='\t')

/var/folders/hf/frc4nxb532j7gqr3_36l6kmh0000gn/T/ipykernel_5892/1121805674.py:10: DtypeWarning: Columns (0,1,2,5,6,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252

## Comorbidities Anx and ODD/CD

In [9]:
for col in range(dsm4.shape[1]):
    print(col, dsm4.iloc[0,col])

0 collection_id
1 scid01_id
2 dataset_id
3 The NDAR Global Unique Identifier (GUID) for research subject
4 Subject ID how it's defined in lab/project
5 Age in months at the time of the interview/test/sampling/imaging.
6 Date on which the interview/genetic test/sampling/imaging/biospecimen was completed. MM/DD/YYYY
7 Sex of subject at birth
8 Bipolar I Lifetime Prevalence
9 Bipolar I Episode Pattern
10 Bipolar I Rapid Cycling
11 Bipolar I Season Pattern
12 Bipolar I Remission Status
13 Bipolar I Past Month
14 Bipolar I Current Episode
15 Bipolar I Depressive Episode Type
16 Bipolar I Current Severity
17 Bipolar II Lifetime Prevalence
18 Bipolar II Rapid Cycling
19 Bipolar II Season Pattern
20 Bipolar II Remission Status
21 Bipolar II Past Month
22 Bipolar II Current Episode
23 Bipolar II Depressive Episode Type
24 Bipolar II Current Severity
25 Other Bipolar Lifetime Prevalence
26 Other Bipolar Current Episode
27 Other Bipolar Current Episode Type
28 Major Depressive Disorder Lifetime P

In [10]:
# for i in range(96, 150):
#     if dsm4.iloc[:, i].dropna().shape[0] > 1:
#         print(i,dsm4.iloc[0, i], dsm4.iloc[:, i].dropna().shape)

In [11]:
# for i in range(96, 150):
#     if diags1.iloc[:, i].dropna().shape[0] > 1:
#         print(i,diags1.iloc[0, i], diags1.iloc[:, i].dropna().shape)

In [12]:
files_diags = ['diagpsx01', 'diagpsx_p201' ,'diagpsx_p301' ,'diagpsx_p401' ,'diagpsx_p501', 'diagpsx_p601', 'diagpsx_p701']

In [13]:
# diags = []
# for file in files_diags:
#     diags.append( pd.read_csv(Path(data_root, file +'.txt'), delimiter='\t'))

/var/folders/hf/frc4nxb532j7gqr3_36l6kmh0000gn/T/ipykernel_5892/3736075964.py:3: DtypeWarning: Columns (0,1,2,6,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,25

In [14]:
# for j, file in zip(range(len(diags)), diags):
#     for i, col in zip(range(file.shape[1]), file.columns): 
#         if file.iloc[:,i].dropna().shape[0] > 1 and (file.iloc[0, i].rfind('anx') != -1 or  file.iloc[0, i].rfind('cond') != -1 or file.iloc[0, i].rfind('opp') != -1 or file.iloc[0, i].rfind('pho') != -1 ) : 
#             print(i,col, file.iloc[0, i], file.iloc[:,i].dropna().shape, files_diags[j])

19 pso Social phobia (13093,) diagpsx01
20 psoi Social phobia with impairment (12903,) diagpsx01
21 pag Agoraphobia (13086,) diagpsx01
22 pagi Agoraphobia with impairment (14475,) diagpsx01
25 pga Generalized anxiety disorder (13081,) diagpsx01
26 pgai Generalized anxiety disorder with impairment (14459,) diagpsx01
27 poa Overanxious disorder (5173,) diagpsx01
28 poai Overanxious disorder with impairment (5173,) diagpsx01
30 psa Separation anxiety disorder (9740,) diagpsx01
31 psai Separation anxiety disorder with impairment (9741,) diagpsx01

In [15]:
diags1.iloc[:,67].dropna().shape #conduct disorder or odd criteria 

(5259,)

In [16]:
baseline_vars = var_dict.get_baseline_vars()
baseline_vars

['src_subject_id',
 'interview_date',
 'interview_age',
 'sex',
 'site',
 'days_baseline',
 'relationship']

In [94]:
var_oddrcd =[ 'cdorodd']
vars_anx = ['pso', 'psoi', 'pag', 'pagi', 'pga', 'pgai' ,'psa', 'psai'] #poa : overanxious disorder: see if included in alaysis 
# for now remove poa because lots of mssing data in comparison 
vars_comorb = np.concatenate((baseline_vars, var_oddrcd, vars_anx))
comorb__ = diags1[vars_comorb]
comorb_ = pd.merge(comorb__, trt_name, how='inner', on='src_subject_id')

In [95]:
vars_comorb_types = np.concatenate((var_oddrcd, vars_anx))
vars_comorb_type_dict = dict(zip(vars_comorb_types, ['category'] * len(vars_anx)))

comorb= prep.set_dtypes(comorb_, vars_comorb_type_dict, set_baseline=True)
comorb.dtypes

Dropping rows containing NaN. Old shape:  (14911, 17), new shape : (5091, 17)
Conversion encountered a problem. Attempt to drop description line.
Success


/Users/alina/Desktop/MIT/code/ADHD/MTA/helper/prep.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/alina/Desktop/MIT/code/ADHD/MTA/helper/prep.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if 'trtname' in df.columns:
/Users/alina/Desktop/MIT/code/ADHD/MTA/helper/prep.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

src_subject_id            object
interview_date    datetime64[ns]
interview_age              int64
sex                     category
site                    category
days_baseline              int64
relationship               int64
cdorodd                 category
pso                     category
psoi                    category
pag                     category
pagi                    category
pga                     category
pgai                    category
psa                     category
psai                      object
trtname                 category
dtype: object

## Public assistance 

In [24]:
services

,collection_id,scapijus01_id,dataset_id,subjectkey,src_subject_id,interview_date,interview_age,sex,monthsbl,daystrt,...,glhitnum,glhitpc,glalthrs,glaltnm,glaladtn,glalldtn,glalbetn,version_form,asssca,collection_title
0,collection_id,scapijus01_id,dataset_id,The NDAR Global Unique Identifier (GUID) for r...,Subject ID how it's defined in lab/project,Date on which the interview/genetic test/sampl...,Age in months at the time of the interview/tes...,Sex of subject at birth,Actual number of months from Baseline,Parent SNAP or CAARS Days from Baseline used a...,...,Number of times during year received most inte...,Percent of year received most intensive therapy,All therapies combined: total amount of hours ...,All therapies combined: number of therapies re...,All therapies combined: number of therapies re...,All therapies combined: number of therapies re...,All therapies combined: number of therapies re...,Form used/assessment name,Assessment point,collection_title
1,2155,83322,9363,NDAR_INVDW676PU1,P1561,02/25/2002,177,F,80,2454,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SCAPIMON,84,Multimodal Treatment Study of Children With ADHD
2,2155,83323,9363,NDAR_INVDW676PU1,P1561,01/06/2003,187,F,90,2769,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SCAPIMON,96,Multimodal Treatment Study of Children With ADHD
3,2155,83324,9363,NDAR_INVDW676PU1,P1561,11/18/2003,197,F,101,3085,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SCAPIMON,108,Multimodal Treatment Study of Children With ADHD
4,2155,83325,9363,NDAR_INVDW676PU1,P1561,09/03/2004,207,F,110,3375,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SCAPIMON,120,Multimodal Treatment Study of Children With ADHD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27304,2155,83317,9364,NDAR_INVXP709VPA,P1844,01/22/2000,129,M,69,1734,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SCAPIPMD,72,Multimodal Treatment Study of Children With ADHD
27305,2155,83318,9364,NDAR_INVXP709VPA,P1844,02/01/2001,141,M,80,2110,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SCAPIPMD,84,Multimodal Treatment Study of Children With ADHD
27306,2155,83319,9364,NDAR_INVXP709VPA,P1844,01/21/2002,153,M,92,2464,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SCAPIPMD,96,Multimodal Treatment Study of Children With ADHD
27307,2155,83320,9364,NDAR_INVXP709VPA,P1844,01/10/2003,165,M,104,2818,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SCAPIPMD,108,Multimodal Treatment Study of Children With ADHD


In [25]:
# file = services
# for i, col in zip(range(file.shape[1]), file.columns): 
#     if file.iloc[:,i].dropna().shape[0] > 1 :#and (file.iloc[0, i].rfind('anx') != -1 or  file.iloc[0, i].rfind('cond') != -1 or file.iloc[0, i].rfind('opp') != -1 or file.iloc[0, i].rfind('pho') != -1 ) : 
#         print(i,col, file.iloc[0, i], file.iloc[:,i].dropna().shape)

In [26]:
demog_file = 'demgr01.txt'
demog = pd.read_csv(Path(data_root, demog_file), delimiter= '\t')


In [28]:
#rep.print_nonNaN_shapes(demog)

In [97]:
assist_vars_ = ['demo61']
assist_vars_dtype = ['category'] # change when found out what the values are, now 1 and 2
assist_vars_dtype_dict = dict(zip(assist_vars_, assist_vars_dtype))

assist_vars = np.concatenate((baseline_vars, assist_vars_))
assist__ = demog[assist_vars]
assist_ = pd.merge(assist__, trt_name, how='inner', on ='src_subject_id')
assist = prep.set_dtypes(assist_, assist_vars_dtype_dict, set_baseline=True)

Dropping rows containing NaN. Old shape:  (4077, 9), new shape : (2056, 9)
Conversion encountered a problem. Attempt to drop description line.
Success


/Users/alina/Desktop/MIT/code/ADHD/MTA/helper/prep.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/alina/Desktop/MIT/code/ADHD/MTA/helper/prep.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if 'trtname' in df.columns:
/Users/alina/Desktop/MIT/code/ADHD/MTA/helper/prep.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

In [98]:
assist.dtypes

src_subject_id            object
interview_date    datetime64[ns]
interview_age              int64
sex                     category
site                    category
days_baseline              int64
relationship               int64
demo61                  category
trtname                 category
dtype: object

## Previous medication


In [31]:
#prep.print_nonNaN_shapes(meds)

In [32]:
health_qst_file  = 'health01.txt'
health_qst = pd.read_csv(Path(data_root, health_qst_file), delimiter='\t')


/var/folders/hf/frc4nxb532j7gqr3_36l6kmh0000gn/T/ipykernel_5892/1521438333.py:2: DtypeWarning: Columns (0,1,2,6,8,9,10,11,12,13,14,15,17,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,39,41,43,45,47,49,50,51,52,53,54,55,56,58,59,60,61,63,64,66,68,70,71,73,75,76,77,78,80,82,84,85,86,87,88,89,91,93,95,97,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270

In [33]:
#prep.print_nonNaN_shapes(health_qst, contains='med')

In [34]:
#prep.print_nonNaN_shapes(demog, contains='med')

119 d13x Amount of debt currently owed medical bills (201,)


In [92]:
prev_med_vars_ = ['hi_24']
prev_med_dytpes = ['category']
prev_med_dtype_dict = dict(zip(prev_med_vars_, prev_med_dytpes))
prev_med_vars = np.concatenate((baseline_vars, prev_med_vars_))
prev_med__ = health_qst[prev_med_vars]
prev_med_ = pd.merge(prev_med__, trt_name, how='inner', on = 'src_subject_id')

prev_med= prep.set_dtypes(prev_med_, prev_med_dtype_dict, set_baseline=True)
prev_med.dtypes

Dropping rows containing NaN. Old shape:  (2828, 9), new shape : (2013, 9)
Conversion encountered a problem. Attempt to drop description line.
Success


/Users/alina/Desktop/MIT/code/ADHD/MTA/helper/prep.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/alina/Desktop/MIT/code/ADHD/MTA/helper/prep.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if 'trtname' in df.columns:
/Users/alina/Desktop/MIT/code/ADHD/MTA/helper/prep.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

src_subject_id            object
interview_date    datetime64[ns]
interview_age              int64
sex                     category
site                    category
days_baseline              int64
relationship               int64
hi_24                   category
trtname                 category
dtype: object

## Initial acceptance 

In [38]:
expect = pd.read_csv(Path(data_root, expectations_file), delimiter='\t')
initial_sat = pd.read_csv(Path(data_root, initial_percieved_satisfaction_file), delimiter='\t')

In [39]:
#prep.print_nonNaN_shapes(expect)

0 collection_id collection_id (5069,)
1 expect01_id expect01_id (5069,)
2 dataset_id dataset_id (5069,)
3 subjectkey The NDAR Global Unique Identifier (GUID) for research subject (5069,)
4 src_subject_id Subject ID how it's defined in lab/project (5069,)
5 interview_date Date on which the interview/genetic test/sampling/imaging/biospecimen was completed. MM/DD/YYYY (5069,)
6 interview_age Age in months at the time of the interview/test/sampling/imaging. (5069,)
7 sex Sex of subject at birth (5069,)
8 days_baseline Days since baseline (5069,)
9 assbdic Assessment Point (5069,)
10 relationship Relationship of respondent to individual (5069,)
11 actbdic Active status (5069,)
12 site Site (5069,)
13 sjtyp Subject type (5069,)
14 exp1 Suppose you ask your child to clean up his/her room. Will your child do it? (5069,)
15 exp2 Imagine you are talking on the telephone and you ask your child to stop interrupting you. Will your child stop? (5064,)
16 exp3 Suppose you ask your child to behave dur

In [40]:
expect['actbdic'].unique()

array(['Active status', '0', '1', '2', '3'], dtype=object)

In [41]:
#prep.print_nonNaN_shapes(initial_sat)

0 collection_id collection_id (580,)
1 debrief01_id debrief01_id (580,)
2 dataset_id dataset_id (580,)
3 subjectkey The NDAR Global Unique Identifier (GUID) for research subject (580,)
4 src_subject_id Subject ID how it's defined in lab/project (580,)
5 interview_date Date on which the interview/genetic test/sampling/imaging/biospecimen was completed. MM/DD/YYYY (580,)
6 interview_age Age in months at the time of the interview/test/sampling/imaging. (580,)
7 sex Sex of subject at birth (580,)
8 days_baseline Days since baseline (580,)
9 assbdic Assessment Point (580,)
10 relationship Relationship of respondent to individual (265,)
11 actbdic Active status (580,)
12 site Site (580,)
13 sjtyp Subject type (580,)
14 d2dresp Which of the following choices best describes the responses of the parent(s) to randomization? (462,)
15 d2dliki Which of the following reflects your judgment about the likelihood of this family adhering to the protocol and remaining committed to the study? (461,)
16 r

In [42]:
print(initial_sat['d2dresp'].unique())

['Which of the following choices best describes the responses of the parent(s) to randomization?'
 nan '3' '4' '6' '5' '1' '2']


In [43]:
end_treat_file = 'debrief1401.txt'
end_treat = pd.read_csv(Path(data_root, end_treat_file), delimiter='\t')

In [44]:
#prep.print_nonNaN_shapes(end_treat)

0 collection_id collection_id (434,)
1 debrief1401_id debrief1401_id (434,)
2 dataset_id dataset_id (434,)
3 subjectkey The NDAR Global Unique Identifier (GUID) for research subject (434,)
4 src_subject_id Subject ID how it's defined in lab/project (434,)
5 interview_date Date on which the interview/genetic test/sampling/imaging/biospecimen was completed. MM/DD/YYYY (285,)
6 interview_age Age in months at the time of the interview/test/sampling/imaging. (285,)
7 sex Sex of subject at birth (434,)
8 days_baseline Days since baseline (285,)
9 assbdic Assessment Point (434,)
10 site Site (434,)
11 sjtyp Subject type (434,)
12 trtname Treatment group (434,)
13 eotcat End of treatment category (432,)
14 db1 Local Attention Deficit/Hyperactivity Disorder Parent Group (e.g., ChADD., LDA) (213,)
15 db2 Other Parent Support Group (e.g., AMI, MHA) (11,)
16 db3 Parenting/Marriage Classes taught thru the community (church/synagogue, etc.) (2,)
17 db4 Other Self-Help Groups, like AA or NA (8,)
18 d

In [101]:
accept_vars_ = ['d2dresp']
accept_vars_dtypes = ['category']
accept_vars_dtypes_dict = dict(zip(accept_vars_, accept_vars_dtypes))
accept_vars = np.concatenate((baseline_vars, accept_vars_))

In [102]:
accept__ = initial_sat[accept_vars]
accept_ = pd.merge(accept__, trt_name, how='inner', on='src_subject_id')
accept = prep.set_dtypes(accept_, accept_vars_dtypes_dict, set_baseline=True)

Dropping rows containing NaN. Old shape:  (580, 9), new shape : (264, 9)
Conversion encountered a problem. Attempt to drop description line.
Success


/Users/alina/Desktop/MIT/code/ADHD/MTA/helper/prep.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/alina/Desktop/MIT/code/ADHD/MTA/helper/prep.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if 'trtname' in df.columns:
/Users/alina/Desktop/MIT/code/ADHD/MTA/helper/prep.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pand

In [103]:
accept.dtypes

src_subject_id            object
interview_date    datetime64[ns]
interview_age              int64
sex                     category
site                    category
days_baseline              int64
relationship               int64
d2dresp                 category
trtname                 category
dtype: object

### Adherence - missing 

In [104]:
print(comorb.shape, prev_med.shape, assist.shape, accept.shape)
print(comorb.dropna().shape, prev_med.dropna().shape, assist.dropna().shape, accept.dropna().shape)
print(comorb['src_subject_id'].unique().shape, prev_med['src_subject_id'].unique().shape, assist['src_subject_id'].unique().shape, accept['src_subject_id'].unique().shape)

(5090, 17) (2012, 9) (2055, 9) (263, 9)
(5090, 17) (2012, 9) (2055, 9) (263, 9)
(866,) (740,) (747,) (263,)


## Save 

In [107]:
comorb.to_csv(Path(data_derived, 'comorb.csv'))
prev_med.to_csv(Path(data_derived, 'prev_med.csv'))
assist.to_csv(Path(data_derived, 'assist.csv'))
accept.to_csv(Path(data_derived, 'accept.csv'))
